In [1]:
# === setup_bookdown.R ===
Sys.setlocale("LC_CTYPE", "zh_TW.UTF-8")
setwd("/Users/iw/Documents/NTU/1132/1132_German_Legal")

# 📁 設定章節來源目錄（從當前資料夾開始遞迴找所有 .md / .Rmd）
library(stringr)

# # --- STEP 1: 修復中間誤用的 ---
# replace_unsafe_hr <- function(file) {
#   lines <- readLines(file, warn = FALSE)
#   yaml_start <- which(lines == "---")[1]
#   yaml_end   <- which(lines == "---")[2]
  
#   # 沒有 YAML 區塊，直接處理整篇
#   if (is.na(yaml_start) || is.na(yaml_end)) {
#     lines_fixed <- gsub("^---$", "***", lines)
#   } else {
#     # 修正 YAML 區塊外的 --- 為 ***
#     lines_fixed <- c(
#       lines[1:yaml_end],
#       gsub("^---$", "***", lines[(yaml_end + 1):length(lines)])
#     )
#   }
  
#   writeLines(lines_fixed, file)
#   message("✅ 已修正 `---` 為 `***` ：", file)
# }


[1] "zh_TW.UTF-8"

In [2]:

# # --- STEP 2: 掃描並修正所有 .md 檔案 ---
# all_md <- list.files(".", pattern = "\\.md$", recursive = TRUE, full.names = TRUE)
# invisible(lapply(all_md, replace_unsafe_hr))



In [3]:


# --- STEP 3: 建立 _bookdown.yml ---
all_files <- list.files(".", pattern = "\\.R?md$", recursive = TRUE, full.names = TRUE)


# 1. 排除開頭是 "_" 的檔案
valid_files <- all_files[!grepl("^_", basename(all_files))]

# 2. 排除資料夾中有任何一層是以 "_" 開頭的檔案
valid_files <- valid_files[!grepl("/_", valid_files)]

# 3. 排除 README 類型的檔案
valid_files <- valid_files[!basename(valid_files) %in% c("README.md", "README.Rmd")]

valid_files <- sort(valid_files)
# 將 index.Rmd 放最前面
valid_files <- c("index.Rmd", setdiff(valid_files, "index.Rmd"))

# YAML 內容
yaml_lines <- c(
  'book_filename: "1132-Readings-of-German-Jurisprudence.Rmd"',
  'clean: [packages.bib, bookdown.bbl]',
  'delete_merged_file: true',
  'language:',
  '  label:',
  '    fig: "圖 "',
  '    tab: "表 "',
  '  ui:',
  '    edit: "編輯"',
  '    chapter_name: ["第 ", " 章"]',
  'output_dir: "docs"',
  'rmd_subdir: true',
  'rmd_files:',
  paste0("  - ", valid_files)
)

writeLines(yaml_lines, "_bookdown.yml")
cat("\n✅ _bookdown.yml 產生成功，共 ", length(valid_files), " 個章節！\n")

# --- STEP 4: 選擇性立即渲染書籍 ---
bookdown::render_book("index.Rmd", "bookdown::gitbook")


✅ _bookdown.yml 產生成功，共  74  個章節！




processing file: 1132-Readings-of-German-Jurisprudence.Rmd



1/1


output file: 1132-Readings-of-German-Jurisprudence.knit.md




/usr/local/bin/pandoc +RTS -K512m -RTS 1132-Readings-of-German-Jurisprudence.knit.md --to html4 --from markdown+autolink_bare_uris+tex_math_single_backslash --output 1132-Readings-of-German-Jurisprudence.html --lua-filter /Library/Frameworks/R.framework/Versions/4.4-arm64/Resources/library/bookdown/rmarkdown/lua/custom-environment.lua --lua-filter /Library/Frameworks/R.framework/Versions/4.4-arm64/Resources/library/rmarkdown/rmarkdown/lua/pagebreak.lua --lua-filter /Library/Frameworks/R.framework/Versions/4.4-arm64/Resources/library/rmarkdown/rmarkdown/lua/latex-div.lua --lua-filter /Library/Frameworks/R.framework/Versions/4.4-arm64/Resources/library/rmarkdown/rmarkdown/lua/table-classes.lua --lua-filter /Library/Frameworks/R.framework/Versions/4.4-arm64/Resources/library/rmarkdown/rmarkdown/lua/anchor-sections.lua --metadata-file /var/folders/kx/yl3w6mcd1g7767kzzplxnd380000gn/T//RtmpHkjLnp/file856f5f7e5081 --wrap preserve --standalone --section-divs --table-of-contents --toc-depth 3 -


Output created: docs/index.html



[1] "/Users/iw/Documents/NTU/1132/1132_German_Legal/docs/index.html"

In [4]:
# files <- list.files(".", pattern = "\\.R?md$", recursive = TRUE, full.names = TRUE)
# for (f in files) {
#   lines <- readLines(f, warn = FALSE)
#   if (!any(grepl("^# ", lines))) {
#     message("⚠️ 檔案缺少章節標題（# 開頭）➡ ", f)
#   }
# }

In [5]:
# # === add_anchor_to_headers.R ===
# library(stringi)

# # 轉換中文標題 ➜ 合法 GitBook ID slug
# slugify <- function(text) {
#   text <- stringi::stri_trans_general(text, "Latin-ASCII") # 轉換變成類拼音/英文
#   text <- tolower(text)
#   text <- gsub("[^a-z0-9]+", "-", text)       # 非字母數字轉成 -
#   text <- gsub("(^-|-$)", "", text)           # 去頭尾的 -
#   return(text)
# }

# # 為章節標題補上 {#slug}
# add_anchors <- function(file) {
#   lines <- readLines(file, warn = FALSE)
#   modified <- FALSE
  
#   lines <- lapply(lines, function(line) {
#     if (grepl("^# ", line) && !grepl("\\{#.*\\}", line)) {
#       title <- gsub("^# +", "", line)
#       title_clean <- gsub("\\*+", "", title)                  # 去除粗體符號
#       title_clean <- gsub("[（(].*?[）)]", "", title_clean)   # 去除括號中的說明
#       slug <- slugify(title_clean)
#       line <- paste0(line, " {#", slug, "}")
#       modified <<- TRUE
#     }
#     return(line)
#   })
  
#   if (modified) {
#     writeLines(unlist(lines), file)
#     message("✅ 已補 anchor：", file)
#   } else {
#     message("☑️ 已有 anchor 或無標題：", file)
#   }
# }

# # 處理所有 .md / .Rmd
# files <- list.files(".", pattern = "\\.R?md$", recursive = TRUE, full.names = TRUE)
# lapply(files, add_anchors)